In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
#Holiday,Season,Month,Week 
feature = ['Holiday', 'Season', 'Month', 'Week', 'Quantity']
result = my_LSTM(feature)
predict_LSTM = result[0]
actual_result = result[1]

In [ ]:
def my_LSTM(feature):
    # Load the dataset
    df = pd.read_csv('sale/sale.csv')  # Replace 'your_dataset.csv' with the actual file path or URL

    #seperate dates for future plotting
    dates = pd.to_datetime(df['Date'])

    # Extract the relevant features
    data = df[feature].values

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)

    # Split into input (features) and output (demand) variables
    features = data_scaled[:, :-1]
    demand = data_scaled[:, -1]

    #Split the dataset into training and testing sets
    # Determine the train-test split ratio
    train_ratio = 0.8
    train_size = int(len(data) * train_ratio)

    # Split the data into train and test sets
    train_features = features[:train_size]
    train_demand = demand[:train_size]
    train_dates = dates[:train_size]
    test_features = features[train_size:]
    test_demand = demand[train_size:]
    test_dates = dates[train_size:]

    #Reshape the input data for LSTM
    # Reshape the features data to match LSTM input shape (samples, time steps, features)
    n_features = features.shape[1]
    train_features = train_features.reshape((train_features.shape[0], 1, n_features))
    test_features = test_features.reshape((test_features.shape[0], 1, n_features))

    #Build and train the LSTM model
    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(train_features, train_demand, epochs=50, batch_size=16, verbose=1)

    #Make predictions on the test set
    # Generate predictions on the test set
    predictions = model.predict(test_features)

    # Inverse scale the predictions and actual demand
    predictions = scaler.inverse_transform(np.concatenate((test_features.reshape(test_features.shape[0], -1), predictions), axis=1))[:, -1]
    actual_demand = scaler.inverse_transform(np.concatenate((test_features.reshape(test_features.shape[0], -1), test_demand.reshape(-1, 1)), axis=1))[:, -1]

    return predictions,actual_demand

In [3]:
cost_per_unit = 352
selling_price_per_unit = 700
min_order_quantity = 10
max_inventory = 20
lead_time = 3
safety_stock = 0  # Set the initial safety stock to 0


In [8]:
def calculate_eoq(forecasted_demand, ordering_cost_per_order, holding_cost_per_unit):
    total_demand = sum(forecasted_demand)
    eoq = np.sqrt((2 * total_demand * ordering_cost_per_order) / holding_cost_per_unit)
    return eoq

# Example usage:
forecasted_demand = [5, 6, 6, 12]  # Adjust with your specific forecasted demand for each week
ordering_cost_per_order = 50  # Adjust as per your specific ordering cost
holding_cost_per_unit = 5  # Adjust as per your specific holding cost

eoq = calculate_eoq(forecasted_demand, ordering_cost_per_order, holding_cost_per_unit)
print("EOQ:", eoq)


EOQ: 24.08318915758459


In [11]:
import math

def calculate_optimal_order_quantity(demand, ordering_cost, holding_cost, max_inventory):
    # Calculate the EOQ formula
    eoq = math.sqrt((2 * demand * ordering_cost) / holding_cost)

    # Adjust the EOQ based on the maximum inventory constraint
    optimal_order_quantity = min(eoq, max_inventory)

    return optimal_order_quantity

# Given data
forecasted_demand = [5, 6]
ordering_cost = 500
holding_cost = 5
max_inventory = 20

# Calculate the total demand
total_demand = sum(forecasted_demand)

# Calculate the optimal order quantity
optimal_order_quantity = calculate_optimal_order_quantity(total_demand, ordering_cost, holding_cost, max_inventory)

print("Optimal Order Quantity:", optimal_order_quantity)


Optimal Order Quantity: 20
